In [1]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage # type: ignore
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_neo4j import Neo4jGraph
from langchain.document_loaders import WikipediaLoader, TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
)

In [2]:
# os.environ["OPENAI_API_KEY"] = "sk-9f91f7f5e8eb4dfabb71c9df5f72e7d2"
# os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"
os.environ["NEO4J_URI"] = "neo4j+s://2656c53b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "kYiyaz3o9-JVrpGqJU39lEFvvb5x9jtfKg3CiO2GYIo"

graph = Neo4jGraph()

In [3]:
# Read the wikipedia article
raw_documents = TextLoader("11--2023年广东省普通高校专升本考试专用教材·教育理论_parsed_output.txt").load()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=2024, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [4]:
# 文本分块----------------------------------------------------------------------
import hanlp

document = TextLoader("edu.txt").load()

# 单任务模型，分词，token的计数是计算词，包括标点符号。
tokenizer = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)

# 划分段落。
def split_into_paragraphs(text):
    return text.split('\n')

# 判断token是否为句子结束符，视情况再增加。
def is_sentence_end(token):
    return token in ['。', '！', '？']

# 向后查找到句子结束符，用于动态调整chunk划分以保证chunk以完整的句子结束
def find_sentence_boundary_forward(tokens, chunk_size):
    end = len(tokens)  # 默认的end值设置为tokens的长度
    for i in range(chunk_size, len(tokens)):  # 从chunk_size开始向后查找
        if is_sentence_end(tokens[i]):
            end = i + 1  # 包含句尾符号
            break
    return end  

# 从位置start开始向前寻找上一句的句子结束符，以保证分块重叠的部分从一个完整的句子开始。
def find_sentence_boundary_backward(tokens, start):
    for i in range(start - 1, -1, -1):
        if is_sentence_end(tokens[i]):
            return i + 1  # 包含句尾符号
    return 0  # 找不到

# 文本分块，文本块的参考大小为chunk_size，文本块之间重叠部分的参考大小为overlap。
# 为了保证文本块之间重叠的部分及文本块末尾截断的部分都是完整的句子，
# 文本块的大小和重叠部分的大小都是根据当前文本块的内容动态调整的，是浮动的值。
def chunk_text(text, chunk_size=2000, overlap=100):
    if chunk_size <= overlap:  # 参数检查
        raise ValueError("chunk_size must be greater than overlap.")
    # 先划分为段落，段落保存了语义上的信息，整个段落去处理。  
    paragraphs = split_into_paragraphs(text)
    chunks = []
    buffer = []
    # 逐个段落处理
    i = 0
    while i < len(paragraphs):
        # 注满buffer，直到大于chunk_szie，整个段落读入，段落保存了语义上的信息。
        while len(buffer) < chunk_size and i < len(paragraphs):
            tokens = tokenizer(paragraphs[i])
            buffer.extend(tokens)
            i += 1
        # 当前buffer分块
        while len(buffer) >= chunk_size:
            # 保证从完整的句子处截断。
            end = find_sentence_boundary_forward(buffer, chunk_size)
            chunk = buffer[:end]
            chunks.append(chunk)  # 保留token的状态以便后面计数
            # 保证重叠的部分从完整的句子开始。
            start_next = find_sentence_boundary_backward(buffer, end - overlap)
            if start_next==0:  # 找不到了上一句的句子结束符，调整重叠范围再找一次。
                start_next = find_sentence_boundary_backward(buffer, end-1)
            if start_next==0:  # 真的找不到，放弃块首的完整句子重叠。
                start_next = end - overlap
            buffer=buffer[start_next:]
        
    if buffer:  # 如果缓冲区还有剩余的token
        # 检查一下剩余部分是否已经包含在最后一个分块之中，它只是留作块间重叠。
        last_chunk = chunks[len(chunks)-1]
        rest = ''.join(buffer)
        temp = ''.join(last_chunk[len(last_chunk)-len(rest):])
        if temp!=rest:   # 如果不是留作重叠，则是最后的一个分块。
            chunks.append(buffer)
    
    return chunks


# 使用自定义函数进行分块
# for file_content in document:
    # print("文件名:", file_content[0])
    # chunks = chunk_text(file_content[1], chunk_size=500, overlap=50)
    # file_content.append(chunks)
    
# 打印分块结果
# for file_content in document:
#     print(f"File: {file_content[0]} Chunks: {len(file_content[2])}")
#     for i, chunk in enumerate(file_content[2]):
#         print(f"Chunk {i+1}: {len(chunk)} tokens.")

# chunks = chunk_text(document, chunk_size=2000, overlap=100)
print(document)

[Document(metadata={'source': 'edu.txt'}, page_content='第一部分 教育学 小库导学小库课堂 第一节教育及其产生与发展 第二节教育学及其产生与发展 13小库考场·· 25小库答案…· 26\n第二章 教育的基本规律 27 小库导学 27 小库课堂 28 第一节教育与社会的发展 28 第二节 教育与人的发展·· 34 小库考场· 41 小库答案 41\n第三章 教育目的与教育制度 43 小库导学 43 小库课堂 44 第一节教育目的 44 第二节教育制度 52 小库考场· 58 小库答案 59\n第四章 教师与学生 60 小库导学 60 小库课堂 60 第一节 教师 60 第二节 学生 72 第三节‧师生关系 75 小库考场·· 78 小库答案··. 79\n第五章 课程 80 小库导学 80 小库课堂 80 第一节 课程概述 80 第二节 课程目标 84 第三节 课程内容 86 第四节 课程开发、设计与实施 88 第五节 课程评价 90 小库考场 91 小库答案·· 91 韦八早 教子 92 小库导学 92 小库课堂 93 第一节 教学概述 93 第二节 教学过程 94 第三节 教学原则 98 第四节 教学组织形式 102 第五节 教学模式 105 第六节 教学方法与教学策略 108 第七节 教学工作的基本环节 113 第八节 教学评价 117 小库考场 119 小库答案 120\n第七章 德育 122 小库导学 122 小库课堂 122 第一节 德育概述 122 第二节 德育过程 125 第三节 德育原则 127 第四节 德育模式 130 第五节 德育的途径与方法 131 小库考场 135 小库答案 135\n第八章 班级管理与班主任工作 136 小库导学 136 小库课堂 136 第一节班级概述 136 第二节 班级管理 138 第三节 班集体的形成与培养 142 第四节 班主任工作 144 小库考场 148 小库答案 148\n第九章 课外、校外教育 149 小库导学 149 小库课堂 149 第一节课外、校外教育概述 149 第二节学校、家庭、社会三结合教育 152 小库考场 154 小库答案 154\n第十章 教育研究及其方法 155 小库导学 155 小库课堂 155 第一节 教育研究概述 155 第

In [5]:
content = document[0]
content = content.page_content
chunks = chunk_text(content, chunk_size=2000, overlap=100)

In [5]:
print(f"Chunks: {len(chunks)}")
print(f"Chunk 1: {len(chunks[0])} tokens.")
print(chunks[0])

# 每个chunk 组织成一个文档
document = []
for chunk in chunks:
    document.append("".join(chunk))

print(document[0])
print(len(document[0]))

Chunks: 116
Chunk 1: 2008 tokens.
['第一', '部分', '教育学', '小', '库', '导', '学', '小', '库', '课堂', '第一节', '教育', '及其', '产生', '与', '发展', '第二节', '教育学', '及其', '产生', '与', '发展', '13', '小', '库', '考场', '·', '·', '25', '小库', '答案', '…', '·', '26', '第二章', '教育', '的', '基本', '规律', '27 小库', '导学', '27 小库', '课堂', '28', '第一节', '教育', '与', '社会', '的', '发展', '28', '第二节', '教育', '与', '人', '的', '发展', '·', '·', '34 小库', '考场', '·', '41 小库', '答案', '41', '第三章', '教育', '目的', '与', '教育', '制度', '43 小库', '导学', '43 小库', '课堂', '44', '第一节', '教育', '目的', '44', '第二节', '教育', '制度', '52 小库', '考场', '·', '58 小库', '答案', '59', '第四章', '教师', '与', '学生', '60 小库', '导学', '60 小库', '课堂', '60', '第一节', '教师', '60', '第二节', '学生', '72', '第三节', '‧', '师生', '关系', '75 小库', '考场', '·', '·', '78 小库', '答案', '··. 79', '第五章', '课程', '80 小库', '导', '学', '80 小库', '课堂', '80', '第一节', '课程', '概述', '80', '第二节', '课程', '目标', '84', '第三节', '课程', '内容', '86', '第四节', '课程', '开发', '、', '设计', '与', '实施', '88', '第五节', '课程', '评价', '90 小库', '考场', '91 小库', '答案', '·', '·', '91', '韦八早', '教子

In [7]:
from langchain_core.documents import Document
doc = []
for i in range(len(document)):
    doc.append(Document(page_content=document[i]))

print(len(doc))

ValidationError: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value=Document(metadata={'sourc...是强迫行为。\n'), input_type=Document]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [7]:
llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
)
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["知识概念", "章节", "习题", "性质", "人物", "事件"],
    allowed_relationships=[
        ("章节", "包含", "知识概念"),
        ("知识概念", "包含", "知识概念"),
        ("知识概念", "需要先修", "知识概念"),
        ("章节", "需要先修", "章节"),
        ("习题", "考察", "知识概念"),
        ("知识概念", "相关", "性质"),
        ("知识概念", "相关", "事件"),
        ("知识概念", "相关", "人物"),
    ],
    node_properties=True,
    relationship_properties=True,
)

# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(doc)
# Store to neo4j
graph.add_graph_documents(
    graph_documents, 
    baseEntityLabel=True, 
    include_source=True
)